### IMPORTING THE REQUIRED LIBRARIES:

In [1]:
import pandas as pd
import numpy as np
import requests
import matplotlib.pyplot as plt
%matplotlib inline
import sportsipy
import sportsreference
from sportsipy.nfl import boxscore,constants,nfl_utils,player,schedule,roster,teams
from sportsipy.nfl.boxscore import Boxscores, Boxscore

### Getting Data From boxscore
##### 1. data in boxscore is stored in Boxscore as per year and week
##### 2. schedules for predicting is from week 1 for 2021
### Data should be collected for last 2 years for making good model:

In [53]:
#current week to predict :
current_week = 2
current_year=2021
#getting the weeks 
period = { year : list(range(1,1+current_week)) if year == 2021 else list(range(1,25)) for year in [2019,2020,2021] }


##### getting schedule for week , JTK

In [54]:
def get_schedule(year):
    weeks = list(range(1,50))
    schedule_df = pd.DataFrame()
    for w in range(len(weeks)):
        date_string = str(weeks[w]) + '-' + str(year)
        try:
            week_scores = Boxscores(weeks[w],year)
        except:
            continue
        week_games_df = pd.DataFrame()
        for g in range(len(week_scores.games[date_string])):
            game = pd.DataFrame(week_scores.games[date_string][g], index = [0])[['away_name', 'away_abbr','home_name', 'home_abbr','winning_name', 'winning_abbr' ]]
            game['week'] = weeks[w]
            week_games_df = pd.concat([week_games_df,game])
        schedule_df = pd.concat([schedule_df, week_games_df]).reset_index().drop(columns = 'index') 
        schedule_df['year']=year
    return schedule_df 

In [55]:
import time
start_time=time.time()
schedule_df=pd.DataFrame()
for year in period.keys():
    schedule_df=schedule_df.append(get_schedule(year),ignore_index=True)
print('time taken to create schedule_df is %s', (time.time() - start_time))

time taken to create schedule_df is %s 170.21640515327454


In [59]:
period.keys()

dict_keys([2019, 2020, 2021])

In [6]:
schedule_df.shape

(808, 8)

#### Getting Away, Home  datasets with BoxScores:

In [56]:
# len(Boxscores(1,2019).games)

In [57]:
# help(Boxscore('201909080sdg'))

In [58]:
# columns=[x[1:] for x in Boxscore('201909080sdg').__dict__.keys()]

In [89]:
#for every year, every week get away, home data for every game , and scedule data:
def game_data_up_to_week(weeks,year):
    game_df=pd.DataFrame()
    for w in range(len(weeks)):
        try:
            date_string=str(w)+'-'+str(year)
#             print(date_string)
            week_scores=Boxscores(w,year)
            for g in range(len(week_scores.games[date_string])):
                temp=pd.DataFrame(week_scores.games[date_string][g],index=[0])
#                 print(temp.shape)
                temp['week']=weeks[w-1]
                temp['year']=year
                game_df=game_df.append(temp)
        except:
            pass
    return   game_df      
        

#### Getting game_df i.e., daily which game happend and who was opponent

In [90]:
start_time=time.time()
game_df=pd.DataFrame()
for year,weeks in period.items():
    game_df=game_df.append(game_data_up_to_week(weeks,year),ignore_index=True)
print('time taken to create weeks_games_df is %s', (time.time() - start_time))
game_df=game_df.reset_index().drop('index',axis=1)

time taken to create weeks_games_df is %s 31.488192796707153


#### From Boxscores we need to get stats of every match for away team, and home team seperately

In [124]:
def game_statistics(boxscore,columns=[]):
    if columns==[]:
        columns=[x[1:] for x in Boxscore(boxscore).__dict__.keys()]
    stats=pd.DataFrame.from_dict(Boxscore(boxscore).__dict__.values()).T.copy()
    stats.columns=columns
    return stats


,uri,date,time,stadium,attendance,duration,away_name,home_name,winner,winning_name,...,home_yards_from_penalties,home_third_down_conversions,home_third_down_attempts,home_fourth_down_conversions,home_fourth_down_attempts,home_time_of_possession,away_abbr,home_abbr,away_players,home_players
0,201909050chi,"Thursday Sep 5, 2019",8:20pm,Soldier Field,62435,3:07,[[]],[[]],None,None,...,10-107,3-15,3-15,0-2,0-2,28:57,GNB,CHI,[<sportsipy.nfl.boxscore.BoxscorePlayer object...,[<sportsipy.nfl.boxscore.BoxscorePlayer object...


In [102]:
Boxscore('201909050chi')

Boxscore for Green Bay Packers at Chicago Bears (Thursday Sep 5, 2019)

In [125]:
pd.DataFrame.from_dict(Boxscore('201909050chi').__dict__.values(),orient='in')

AttributeError: 'bool' object has no attribute 'lower'